In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import nextbike.preprocessing.Preprocessing as prep

from scipy.stats import zscore
import numpy as np
from vincenty import vincenty 
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
from sklearn.linear_model import Lasso,Ridge, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.preprocessing import PolynomialFeatures,StandardScaler
from sklearn.svm import SVR


In [3]:
df = pd.read_csv('../../data/processed/dortmund_trips.csv', index_col=0)
#df = df.sample(50000)

In [6]:
X = df[["month","weekday","day","hour","minute","latitude_start","longitude_start","area_start",
        "temperature °C","precipitation", "distanceToCentralStation","distanceToUniversity"]]
y = df["trip_duration"]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

# Polynomial Regression - RandomizedSearch

In [35]:
# parameter tuning
poly_reg = PolynomialFeatures(degree = 4)
x_poly = poly_reg.fit_transform(X_train)

ridge = Ridge()
alpha = [1,2,3,4,5,8,10,15,20,30,50]
copy = [True, False]
fit_intercept = [True,False]
max_iter = [10,20,30,40,50,60,70,80,90,100]
solver = ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']

random_grid = {'alpha': alpha,
               'copy_X': copy,
               'fit_intercept': fit_intercept,
               'max_iter': max_iter,
               'solver': solver}

# Random search of parameters
ridge_random = RandomizedSearchCV(estimator = ridge, param_distributions = random_grid, n_iter = 100,
                                  cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the model
ridge_random.fit(x_poly, y_train)
# print results
print(ridge_random.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 28.8min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 66.6min finished


{'solver': 'cholesky', 'max_iter': 40, 'fit_intercept': True, 'copy_X': False, 'alpha': 50}


# Random Forrest Regressor - RandomizedSearchCV

In [5]:
from sklearn.model_selection import RandomizedSearchCV
# random forest model creation
rfc = RandomForestRegressor()
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8,10,15,20,30]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# Random search of parameters
rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the model
rfc_random.fit(X_train, y_train)
# print results
print(rfc_random.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 10.3min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 70.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 132.9min finished


{'n_estimators': 1155, 'min_samples_split': 10, 'min_samples_leaf': 8, 'max_features': 'auto', 'max_depth': 70, 'bootstrap': True}


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# random forest model creation
rfc = RandomForestRegressor()
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8,10,15,20,30]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# Random search of parameters
rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the model
rfc_random.fit(X_train, y_train)
# print results
print(rfc_random.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
